In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Hyperparameters
timesteps = 10  # Number of time steps
num_features = 5  # Number of features
n_epochs = 100  # Number of epochs
learning_rate = 0.001

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(num_features, 50, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(50, 50, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.lstm3 = nn.LSTM(50, 50, batch_first=True)
        self.dropout3 = nn.Dropout(0.2)
        self.lstm4 = nn.LSTM(50, 50, batch_first=True)
        self.dropout4 = nn.Dropout(0.2)
        self.fc = nn.Linear(50, 1)  # Ensures the output is of size [batch_size, 1]
        # self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x, _ = self.lstm3(x)
        x = self.dropout3(x)
        x, _ = self.lstm4(x)
        x = self.dropout4(x)
        x = x[:, -1, :]  # Take the last output, remove this?
        x = self.fc(x)   # Linear layer to map to 1 output
        # x = self.sigmoid(x)
        return x

# GRU Model
class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(num_features, 50, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.gru2 = nn.GRU(50, 50, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.gru3 = nn.GRU(50, 50, batch_first=True)
        self.dropout3 = nn.Dropout(0.2)
        self.gru4 = nn.GRU(50, 50, batch_first=True)
        self.dropout4 = nn.Dropout(0.2)
        self.fc = nn.Linear(50, 1)  # Ensures the output is of size [batch_size, 1]
        # self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x, _ = self.gru1(x)
        x = self.dropout1(x)
        x, _ = self.gru2(x)
        x = self.dropout2(x)
        x, _ = self.gru3(x)
        x = self.dropout3(x)
        x, _ = self.gru4(x)
        x = self.dropout4(x)
        x = x[:, -1, :]  # Take the last output, remove this?
        x = self.fc(x)   # Linear layer to map to 1 output
        # x = self.sigmoid(x)
        return x

In [40]:
import pandas as pd
import numpy as np

#We minmaxed these splits to get (73,9,9) in train/val/test
split = 0.69  # Adjust to allocate space for validation set
val_split = 0.16  # 15% for validation, and implicitly 15% for test due to remaining percentage
sequence_length = 11
normalise = True
batch_size = 100
input_dim = 5
input_timesteps = 10
neurons = 50
epochs = 5
prediction_len = 1
dense_output = 1
drop_out = 0

dataframe = pd.read_csv("data/original_dataset/source_price.csv")
cols = ['Adj Close', 'wsj_mean_compound', 'cnbc_mean_compound', 'fortune_mean_compound', 'reuters_mean_compound']
cols_temp = ['date', 'Adj Close', 'wsj_mean_compound', 'cnbc_mean_compound', 'fortune_mean_compound', 'reuters_mean_compound']

len_dataframe = dataframe.shape[0]

# Split data into train, validation, and test
i_split = int(len(dataframe) * split)
i_val = int(len(dataframe) * (split + val_split))

data_train = dataframe.get(cols).values[:i_split]
data_val = dataframe.get(cols).values[i_split:i_val]
data_test = dataframe.get(cols).values[i_val:]

len_train = len(data_train)
len_val = len(data_val)
len_test = len(data_test)
len_train_windows = None

print('data_train.shape', data_train.shape)
print('data_val.shape', data_val.shape)
print('data_test.shape', data_test.shape)

# Process train data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
window_data = data_windows
win_num = window_data.shape[0]
col_num = window_data.shape[2]
normalised_data = []
record_min = []
record_max = []

# Normalize train data
for win_i in range(win_num):
    normalised_window = []
    for col_i in range(0,1):
      temp_col = window_data[win_i,:,col_i]
      temp_min = min(temp_col)
      record_min.append(temp_min)
      temp_col = temp_col - temp_min
      temp_max = max(temp_col)
      record_max.append(temp_max)
      temp_col = temp_col / temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      normalised_window.append(window_data[win_i,:,col_i])
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data = np.array(normalised_data)
x_train = normalised_data[:, :-1]
y_train = normalised_data[:, -1, [0]]
print('x_train.shape', x_train.shape)
print('y_train.shape', y_train.shape)

# Process validation data
data_windows = []
for i in range(len_val - sequence_length):
    data_windows.append(data_val[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
window_data = data_windows
win_num = window_data.shape[0]
normalised_data = []

# Normalize validation data
for win_i in range(win_num):
    normalised_window = []
    for col_i in range(0,1):
      temp_col = window_data[win_i,:,col_i]
      temp_min = min(temp_col)
      temp_col = temp_col - temp_min
      temp_max = max(temp_col)
      temp_col = temp_col / temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      normalised_window.append(window_data[win_i,:,col_i])
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data = np.array(normalised_data)
x_val = normalised_data[:, :-1]
y_val = normalised_data[:, -1, [0]]
print('x_val.shape', x_val.shape)
print('y_val.shape', y_val.shape)

# Process test data
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
y_test_ori = data_windows[:, -1, [0]]
window_data = data_windows
win_num = window_data.shape[0]
normalised_data = []

# Normalize test data
for win_i in range(win_num):
    normalised_window = []
    for col_i in range(0,1):
      temp_col = window_data[win_i,:,col_i]
      temp_min = min(temp_col)
      temp_col = temp_col - temp_min
      temp_max = max(temp_col)
      temp_col = temp_col / temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      normalised_window.append(window_data[win_i,:,col_i])
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data = np.array(normalised_data)
x_test = normalised_data[:, :-1]
y_test = normalised_data[:, -1, [0]]
print('x_test.shape', x_test.shape)
print('y_test.shape', y_test.shape)


print(x_train[0])

print(y_train[0])


data_train.shape (83, 5)
data_val.shape (19, 5)
data_test.shape (19, 5)
x_train.shape (72, 10, 5)
y_train.shape (72, 1)
x_val.shape (8, 10, 5)
y_val.shape (8, 1)
x_test.shape (8, 10, 5)
y_test.shape (8, 1)
[[ 0.          0.296      -0.1366      0.          0.        ]
 [ 0.2730357   0.          0.         -0.2423      0.        ]
 [ 0.4326822   0.          0.          0.          0.        ]
 [ 0.51015723  0.          0.          0.          0.        ]
 [ 0.48646392  0.          0.          0.          0.        ]
 [ 0.28262597 -0.17253333  0.          0.          0.        ]
 [ 0.73016413 -0.1806      0.          0.          0.        ]
 [ 1.          0.          0.          0.          0.        ]
 [ 0.8365936   0.          0.          0.          0.        ]
 [ 0.79484908  0.          0.          0.          0.        ]]
[0.89488809]


In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# Instantiate models
lstm_model = LSTMModel()
gru_model = GRUModel()

# Define loss and optimizer
criterion = nn.MSELoss()
lstm_optimizer = optim.RMSprop(lstm_model.parameters(), lr=0.0008)
gru_optimizer = optim.RMSprop(gru_model.parameters(), lr=0.0008)

# Convert data to PyTorch tensors and create DataLoader
X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

print(X_train_tensor.shape)
print(y_train_tensor.shape)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training function
def train_model(model, optimizer, criterion, train_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch.view(-1, 1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss/len(train_loader)}')

# Train the LSTM model
print("Training LSTM Model")
train_model(lstm_model, lstm_optimizer, criterion, train_loader, n_epochs)

# Train the GRU model
print("Training GRU Model")
train_model(gru_model, gru_optimizer, criterion, train_loader, n_epochs)

torch.Size([72, 10, 5])
torch.Size([72, 1])
Training LSTM Model
Epoch 1/100, Loss: 0.5322182074189186
Epoch 2/100, Loss: 0.1443455621600151
Epoch 3/100, Loss: 0.13437049090862274
Epoch 4/100, Loss: 0.13468450009822847
Epoch 5/100, Loss: 0.13123217225074768
Epoch 6/100, Loss: 0.13874868750572206
Epoch 7/100, Loss: 0.12576577961444854
Epoch 8/100, Loss: 0.1366221696138382
Epoch 9/100, Loss: 0.13304941952228547
Epoch 10/100, Loss: 0.11117208302021027
Epoch 11/100, Loss: 0.09449929669499398
Epoch 12/100, Loss: 0.11299969553947449
Epoch 13/100, Loss: 0.06886934041976929
Epoch 14/100, Loss: 0.078715780377388
Epoch 15/100, Loss: 0.06360184475779533
Epoch 16/100, Loss: 0.06559820920228958
Epoch 17/100, Loss: 0.0667670987546444
Epoch 18/100, Loss: 0.0811374008655548
Epoch 19/100, Loss: 0.0682803712785244
Epoch 20/100, Loss: 0.07927219718694686
Epoch 21/100, Loss: 0.07804757207632065
Epoch 22/100, Loss: 0.07457466945052146
Epoch 23/100, Loss: 0.07134479209780693
Epoch 24/100, Loss: 0.05982662737

In [42]:
lstm_val_predictions = lstm_model(torch.tensor(x_val, dtype=torch.float32)).detach().numpy()
gru_val_predictions = gru_model(torch.tensor(x_val, dtype=torch.float32)).detach().numpy()

# Combine predictions to form new training data for the meta-learner
meta_X_train = np.concatenate((lstm_val_predictions, gru_val_predictions), axis=1)

print(meta_X_train.shape)

(8, 2)


In [43]:
# Define the meta-learner model
# it's a fully-connect neuralnetwork with three layers; the activation function for this model is the Rectified Linear Unit (ReLu).
# NOTE: The paper doesn't specify the number of neurons in the hidden layers, so I'm basing on the stanford paper
class MetaLearner(nn.Module):
    def __init__(self):
        super(MetaLearner, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 20)
        self.fc4 = nn.Linear(20, 1)
        # self.sigmoid = nn.Sigmoid() 
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        # x = self.fc4(x)
        # x = self.sigmoid(x) #We also asume a sigmoid activation function for the output layer
        return x

In [44]:
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
# LSTM MODEL
batch_size=100;
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=50,batch_size=batch_size)

Epoch 1/50


c:\Users\karlo\scoop\apps\python\current\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.4925
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.4590
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.4271
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.3959
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.3646
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.3327
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.3006
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.2691
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2403
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.2175
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.2059
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.2086
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.2182
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.2222
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.2166
Epoch 16/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/st

In [45]:
from numpy import newaxis
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)
    
print(prediction_seqs)


# In[32]:


#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)


# In[33]:


error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])
    
squaredError = []
absError = []
for val in error:
    squaredError.append(val * val) 
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[[array([0.667582], dtype=float32)], [array([0.5562453], dtype=float32)], [array([0.5648408], dtype=float32)], [array([0.5654941], dtype=float32)], [array([0.5680066], dtype=float32)], [array([0.43610486], dtype=float32)], [array([0.46016136], dtype=float32)], [array([0.41569602], dtype=float32)]]
[2672.4819440593915, 2673.004395392071, 2673.5586309218093, 2673.5835552641925, 2673.6794051757097, 2668.647367818824, 2683.8256670624755, 2690.0092952286873]
MSE [2172.55369241]
accuracy [0.98393374]
mean_error_percent [0.01606626]


In [46]:
from numpy import newaxis
# Train the meta-learner model
meta_model = MetaLearner()
meta_criterion = nn.MSELoss()
meta_optimizer = optim.Adam(meta_model.parameters(), lr=0.008)

meta_X_train_tensor = torch.tensor(meta_X_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

meta_train_dataset = TensorDataset(meta_X_train_tensor, y_val_tensor)
meta_train_loader = DataLoader(meta_train_dataset, batch_size=1, shuffle=False)

train_model(meta_model, meta_optimizer, meta_criterion, meta_train_loader, 50)

prediction_len=1
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

print("multi sequence predict loop")
for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
        print(curr_frame[newaxis,:,:].shape)
        lvl0gru=gru_model(torch.from_numpy(curr_frame[newaxis,:,:]).float())[0]
        lvl0lstm=lstm_model(torch.from_numpy(curr_frame[newaxis,:,:]).float())[0]
        lvl1in=np.array([lvl0gru.detach().numpy(),lvl0lstm.detach().numpy()])
        print(lvl1in)
        lvl1in=torch.from_numpy(lvl1in).float()
        
        lvl1out=meta_model(lvl1in.T)
        print("lvl1out", lvl1out)
        # print("lvl1in shape", lvl1in.shape)
        predicted.append((lvl0gru.detach().numpy() + lvl0lstm.detach().numpy())/2)
        print("predicted", predicted[-1])
        print("true", y_test[i*prediction_len+j])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)
    
print(prediction_seqs)

de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])
    
squaredError = []
absError = []
for val in error:
    squaredError.append(val * val) 
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)




Epoch 1/50, Loss: 0.05640774519270053
Epoch 2/50, Loss: 0.03342993260594085
Epoch 3/50, Loss: 0.028684740311291534
Epoch 4/50, Loss: 0.025240618752036426
Epoch 5/50, Loss: 0.020162962609902024
Epoch 6/50, Loss: 0.014175346193951555
Epoch 7/50, Loss: 0.00963102461683718
Epoch 8/50, Loss: 0.008472580772831861
Epoch 9/50, Loss: 0.008172692522748548
Epoch 10/50, Loss: 0.00780616159318015
Epoch 11/50, Loss: 0.007440271088853478
Epoch 12/50, Loss: 0.007310820368729765
Epoch 13/50, Loss: 0.007259445563249756
Epoch 14/50, Loss: 0.007205244655779097
Epoch 15/50, Loss: 0.007121378777810605
Epoch 16/50, Loss: 0.007170323977334192
Epoch 17/50, Loss: 0.00703939230152173
Epoch 18/50, Loss: 0.006880594108224614
Epoch 19/50, Loss: 0.006770172625692794
Epoch 20/50, Loss: 0.006862823873234447
Epoch 21/50, Loss: 0.006647300157055724
Epoch 22/50, Loss: 0.0064241950276482385
Epoch 23/50, Loss: 0.006578618430467031
Epoch 24/50, Loss: 0.006283667480602162
Epoch 25/50, Loss: 0.006078634978621267
Epoch 26/50, 

In [47]:
#  the test dataset will be input into the sub-models again to produce intermediate test data for the meta-learner. Afterward, the meta-learner will use the intermediate test predictions from the sub-models to make the final predictions.
print(x_test)
lstm_test_predictions = lstm_model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy()
gru_test_predictions = gru_model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy()

meta_X_test = np.concatenate((lstm_test_predictions, gru_test_predictions), axis=1)
meta_X_test_tensor = torch.tensor(meta_X_test, dtype=torch.float32)


meta_test_predictions = meta_model(meta_X_test_tensor).detach().numpy()
print(meta_test_predictions.shape)

de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(meta_test_predictions[i][j]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

# print(meta_test_predictions)
print(y_test_ori)

# Calculate price fluctuations and determine if the price increases (1) or decreases (0)
price_fluctuations = []
for i in range(1, len(de_predicted)):
    if de_predicted[i] > de_predicted[i-1]:
        price_fluctuations.append(1)
    else:
        price_fluctuations.append(0)

print("Price Fluctuations:", price_fluctuations)

price_fluctuations_actual = []
for i in range(1, len(y_test_ori)):
    if y_test_ori[i] > y_test_ori[i-1]:
        price_fluctuations_actual.append(1)
    else:
        price_fluctuations_actual.append(0)

print("Price Fluctuations Actual:", price_fluctuations_actual)

# Calculate accuracy, precision, and recall and f1
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
for i in range(len(price_fluctuations)):
    if price_fluctuations[i] == 1 and price_fluctuations_actual[i] == 1:
        true_positive += 1
    elif price_fluctuations[i] == 0 and price_fluctuations_actual[i] == 0:
        true_negative += 1
    elif price_fluctuations[i] == 1 and price_fluctuations_actual[i] == 0:
        false_positive += 1
    else:
        false_negative += 1

accuracy = (true_positive + true_negative) / len(price_fluctuations)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f1 = 2 * (precision * recall) / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

MSE = np.mean((de_predicted- y_test_ori) ** 2)
print(f'MSE: {MSE}')




[[[ 1.16215416e-02 -1.08273580e-02  7.88724380e-02 -3.33307690e-02
    7.41420580e-02]
  [ 0.00000000e+00  2.02213240e-02  8.19771910e-02  6.92661290e-02
    4.28054450e-02]
  [ 4.23474869e-01  2.60716760e-02  8.56365200e-02  4.35797100e-03
    5.38117060e-02]
  [ 8.37290429e-01 -1.65616280e-02  8.80705360e-02 -1.19285710e-02
    3.54122690e-02]
  [ 9.13405936e-01 -3.79798510e-02  7.17808980e-02 -2.89818180e-02
    3.83915790e-02]
  [ 9.52854430e-01  2.80393160e-02  8.17107320e-02  1.24772700e-03
    5.82385000e-04]
  [ 6.47077624e-01 -2.27106150e-02  7.51553040e-02 -1.34521740e-02
    1.95304100e-03]
  [ 8.27303424e-01  3.01822090e-02  8.28230080e-02 -1.89333330e-02
    1.82742480e-02]
  [ 7.89161754e-01 -2.19021860e-02  7.31291990e-02 -9.77190480e-02
   -2.19987100e-03]
  [ 6.71959238e-01 -4.14457100e-03  7.02473440e-02 -2.83441860e-02
    2.11152410e-02]]

 [[ 0.00000000e+00  2.02213240e-02  8.19771910e-02  6.92661290e-02
    4.28054450e-02]
  [ 4.23474869e-01  2.60716760e-02  8.563